In [1]:
import joblib
import numpy as np
import pandas as pd
import warnings

warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv(r"D:\Ultimate Programming\Data Bases\Machine Learning Datasets\Classification\netflix_customer_churn.csv")

In [3]:
df.head(2)

,customer_id,age,gender,subscription_type,watch_hours,last_login_days,region,device,monthly_fee,churned,payment_method,number_of_profiles,avg_watch_time_per_day,favorite_genre
0,a9b75100-82a8-427a-a208-72f24052884a,51,Other,Basic,14.73,29,Africa,TV,8.99,1,Gift Card,1,0.49,Action
1,49a5dfd9-7e69-4022-a6ad-0a1b9767fb5b,47,Other,Standard,0.70,19,Europe,Mobile,13.99,1,Gift Card,5,0.03,Sci-Fi


In [4]:
df.drop('customer_id', axis=1, inplace=True)

In [5]:
df.head(1)

,age,gender,subscription_type,watch_hours,last_login_days,region,device,monthly_fee,churned,payment_method,number_of_profiles,avg_watch_time_per_day,favorite_genre
0,51,Other,Basic,14.73,29,Africa,TV,8.99,1,Gift Card,1,0.49,Action


In [6]:
print(df.isnull().sum().sum())

0


In [7]:
x = df.drop('churned', axis=1)
y = df['churned']

In [8]:
x.head(1)

,age,gender,subscription_type,watch_hours,last_login_days,region,device,monthly_fee,payment_method,number_of_profiles,avg_watch_time_per_day,favorite_genre
0,51,Other,Basic,14.73,29,Africa,TV,8.99,Gift Card,1,0.49,Action


In [9]:
x.shape

(5000, 12)

In [10]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score

In [12]:
x.columns

Index(['age', 'gender', 'subscription_type', 'watch_hours', 'last_login_days',
       'region', 'device', 'monthly_fee', 'payment_method',
       'number_of_profiles', 'avg_watch_time_per_day', 'favorite_genre'],
      dtype='object')

In [19]:
trf1 = ColumnTransformer(transformers=[
    ('gender', OneHotEncoder(drop='first', sparse_output=False), [1]),
    ('subs_type', OrdinalEncoder(), [2]),
    ('region', OneHotEncoder(drop='first', sparse_output=False), [5]),
    ('device', OrdinalEncoder(), [6]),
    ('pay_meth', OneHotEncoder(drop='first', sparse_output=False), [9]),
    ('fav_genre', OneHotEncoder(drop='first', sparse_output=False), [11])
],remainder='passthrough')

In [21]:
x_sc = trf1.fit_transform(x)
x_sc.shape

(5000, 25)

In [22]:
trf2 = ColumnTransformer(transformers=[
    ('scaled', MinMaxScaler(), slice(0, 25)) 
], remainder='passthrough')

In [28]:
from sklearn.linear_model import LogisticRegression
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

lr = LogisticRegression()

In [29]:
from sklearn.pipeline import make_pipeline

In [30]:
pipeline = make_pipeline(trf1, trf2, lr)

In [31]:
pipeline.fit(x_train, y_train)

ValueError: could not convert string to float: 'Crypto'

In [26]:
from sklearn import set_config
set_config(display='diagram')